In [7]:
!pip install tensorflow gym keras-rl 'gym[atari]' 'gym[accept-rom-license]' opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=d8b3cc73d5d1a1711349aeee23ae1dff48990bb1d37c377e7ae628f64c739cd5
  Stored in directory: /Users/prithvi/Library/Caches/pip/wheels/bc/fc/c6/8aa657c0d2089982f2dabd110efc68c61eb49831fdb7397351
Successfully built AutoROM.accept-rom-license


In [20]:
import gym 
import random
import cv2
import numpy as np

In [9]:
env = gym.make('Breakout-v4', render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n


In [10]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [11]:
env = gym.make('Breakout-v4', render_mode='human')
episodes = 5 
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done: 
        env.render()
        action = random.choice([0,1,2,3])
        n_state, reward, done, truncated, info = env.step(action)
        score += reward
    print("Episode:", episode, "Score:", score)

env.close()

/Users/prithvi/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/Users/prithvi/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0 Score: 2.0
Episode: 1 Score: 3.0
Episode: 2 Score: 0.0
Episode: 3 Score: 3.0
Episode: 4 Score: 0.0


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# Use the Baseline Atari environment because of Deepmind helper functions
env = gym.make("Breakout-v4")
env.seed(seed)

(3444837047, 2669555309)

In [13]:
num_actions = 4


def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(84, 84, 4,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)


# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

In [14]:
from keras.layers import Dense, Activation
from keras.models import Sequential, load_model
from keras.optimizers import Adam
import numpy as np

class replayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions, discrete=False):
        self.mem_size = max_size
        self.input = input_shape
        #dealing with discrete (1 hot) or continous, policy gradients
        self.discrete = discrete 
        self.state_memory = np.zeros(((self.mem_size,) + input_shape))
        self.new_state_memory = np.zeros(((self.mem_size,) + input_shape))
        dtype = np.int8 if self.discrete else np.float32
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.mem_cntr = 0

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state[0]
        self.new_state_memory[index] = state_[0]
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1 - int(done)
        if self.discrete:
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0
            self.action_memory[index] = actions
        else:
            self.action_memory[index] = action 
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        rewards = self.reward_memory[batch]
        actions = self.action_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal
    
def build_dqn(lr, n_actions, input_dims, dcl_dims, fc2_dims):
    model = Sequential([
                Dense(dcl_dims, input_shape=input_dims),
                Activation('relu'),
                Dense(fc2_dims),
                Activation('relu'),
                Dense(n_actions)])


class Agent(object):
    def __init__(self, alpha, gamma, n_actions, epsilon, batch_size, input_dims, epsilon_dec=0.996, epsilon_end=0.01, mem_size=10000, fname='dqn_model.h5'):
        self.action_space = [i for i in range(n_actions)]
        self.n_actions = n_actions
        self.gamma = gamma 
        self.epsilon = epsilon 
        self.epsilon_dec = epsilon_dec
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        self.model_file = fname

        self.memory = replayBuffer(mem_size, input_dims, n_actions, discrete=True)

        self.q_eval = build_dqn(alpha, n_actions, input_dims, 256, 256)

    def remeber(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def chose_action(self, state):
        rand = np.random.random()
        if rand < self.epsilon:
            action = np.random.choice(self.action_space)
        else:
            actions = self.q_eval.predict(state)
            action = np.argmax(actions)

        return action
        
    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return
        
        #sample from the memory buffer, non-sequential memories (avoid correlations in learning)
        state, action, reward, new_state, done = self.memory.sample_buffer(self.batch_size)
        
        #go from one hot encoding to integer
        action_values = np.array(self.action_space, dtype=np.int8)
        action_indices = np.dot(action, action_values)

        q_eval = self.q_eval.predict(state)
        q_next = self.q_eval.predict(new_state)

        q_target = q_eval.copy()


        batch_index = np.arange(self.batch_size, dtype=np.int32)

        #update q targets based on max values of next state
        print("q target is ",q_target)
        print("q next is ", q_next)
        q_target[batch_index, action_indices] = reward + self.gamma * np.max(q_next, axis=1) * done

        #use q_target as target for loss function of q network
        _ = self.q_eval.fit(state, q_target, verbose=0)

        self.epsilon = self.epsilon*self.epsilon_dec if self.epsilon > self.epsilon_min else self.epsilon_min

    def save_model(self):
        self.q_eval.save(self.model_file)

    def load_model(self):
        self.q_eval = load_model(self.model_file)




In [15]:
n_games = 20
agent = Agent(gamma=.99, epsilon=1.0, alpha=.0005, n_actions=4, input_dims=(210, 160, 3) , mem_size=100, batch_size=64, epsilon_end=0.01)

scores = []
eps_history = []

for i in range(n_games):
    done = False
    score = 0
    observation = env.reset()
    while not done: 
        action = agent.chose_action(observation)
        observation_, reward, truncated, done, info = env.step(action)
        score += reward
        agent.remeber(observation, action, reward, observation_, done)
        observation = observation_
        agent.learn()

    eps_history.append(agent.epsilon)
    scores.append(score)

    avg_score = np.mean(scores[max(0, 1 - 100):(i + 1)])
    print("episode ", i, "score %.2f" % score, 'average score %.2f' % avg_score)

    if i % 10 == 0 and i > 0: 
        agent.save_model()


/Users/prithvi/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


AttributeError: 'NoneType' object has no attribute 'predict'

Class For Memory Buffer

In [16]:
class Memory(object):
    def __init__(self, input_shape, num_actions):
        self.states = []
        self.next_states = []
        self.actions = []
        self.rewards = []
        self.terminal = []

    def save_action(self, state, action, reward, next_state, done):
        print("length of state in save actions is ", len(state))
        self.states.append(state)
        self.next_states.append(next_state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.terminal.append(1 - int(done))

    def sample_memory(self):
        return self.states, self.actions, self.rewards, self.next_states, self.terminal
    



Function For Deep Q Network

In [17]:
def init_deep_q_network(input_shape, lr, num_actions, l1_size, l2_size):
    model = Sequential([
        Dense(l1_size, input_shape=input_shape),
        Activation('relu'),
        Dense(l2_size),
        Activation('relu'),
        Dense(num_actions)])
    
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    return model

Class For Learning Agent

In [18]:
class Agent(object):
    def __init__(self, alpha, gamma, actions_amt, epsilon, batch_size, input_shape, epsilon_dec=0.996, epsilon_end=0.01, mem_size=10000, fname='dqn_model.h5'):
        self.alpha = alpha
        self.gamma = gamma
        self.actions_amt = actions_amt
        self.epsilon = epsilon 
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.epislon_dec = epsilon_dec
        self.epsilon_end = epsilon_end
        self.mem_size = mem_size
        self.fname = fname
        self.q_network = init_deep_q_network(input_shape, alpha, actions_amt, 256, 256)
        self.q_network_target = init_deep_q_network(input_shape, alpha, actions_amt, 256, 256)
        self.memory = Memory(input_shape, actions_amt)
        self.loss_function = keras.losses.Huber()
        self.optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)


    def select_action(self, input): 
        if np.random.rand() > epsilon:
            action_values = self.q_network.predict(input)
            action = np.argmax(action_values)
        else:
            action = np.random.choice(num_actions)
        
        return action 
    
    def store_action(self, state, action, reward, next_state, done):
        self.memory.save_action(state, action, reward, next_state, done)

    def learn(self):
        print("in learning call")
        states, actions, rewards, new_states, dones = self.memory.sample_memory()
        for state in states:
            print("shape of state before stack is", np.shape(state[0]))
        states = tf.stack(states)
        actions = tf.stack(actions)
        rewards = tf.stack(rewards)
        new_states = tf.stack(new_states)
        dones = tf.stack(dones)
        new_states = tf.stack(new_states)
        print("new states shape is", np.shape(new_states))
        future_rewards = self.q_network_target.predict(new_states)
        print("after predicting of future rewards")
        updated_qs = rewards + gamma * tf.reduce_max(future_rewards, axis=1)
        masks = tf.one_hot(actions, self.actions_amt)
        print("after making masks")
        with tf.GradientTape() as tape:
            q_values = self.q_network(states)
            q_actions = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
            loss = self.loss_function(updated_qs, q_actions)
        print("after gradient step")
        grads = tape.gradient(loss, self.q_network.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.q_network.trainable_variables))
        self.q_network_target.set_weights(self.q_network.get_weights())

    def save_model(self):
        self.q_network.save(self.model_file)

    def load_model(self):
        self.q_network = load_model(self.model_file)
        

        

Run Learning Loop

In [19]:
n_games = 20
agent = Agent(gamma=.99, epsilon=1.0, alpha=.0005, actions_amt=4, input_shape=(210, 160, 3) , mem_size=100, batch_size=64, epsilon_end=0.01)

scores = []
eps_history = []

for i in range(n_games):
    done = False
    score = 0
    observation = env.reset()
    j = 1
    while not done: 
        action = agent.select_action(observation)
        observation_, reward, truncated, done, info = env.step(action)
        score += reward
        print("in main loop, shape of observation is ", np.shape(observation[0]), "and shape of observation_ is", np.shape(observation_))
        agent.store_action(observation[0], action, reward, observation_, done)
        observation = observation_
        if j % 10 == 0:
            agent.learn()
        j = j + 1

    eps_history.append(agent.epsilon)
    scores.append(score)

    avg_score = np.mean(scores[max(0, 1 - 100):(i + 1)])
    print("episode ", i, "score %.2f" % score, 'average score %.2f' % avg_score)

    if i % 10 == 0 and i > 0: 
        agent.save_model()

in main loop, shape of observation is  (210, 160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  210
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (210, 160, 3)
length of state in save actions is  160
in main loop, shape of observation is  (160, 3) and shape of observation_ is (

InvalidArgumentError: {{function_node __wrapped__Pack_N_10_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [210,160,3] != values[1].shape = [160,3] [Op:Pack] name: stack

Function for resizing frame

In [21]:
def resize_frame(frame):
    frame = frame[50:-5,5:-4]
    frame = np.average(frame,axis = 2)
    frame = cv2.resize(frame,(84,84),interpolation = cv2.INTER_NEAREST)
    frame = np.array(frame,dtype = np.uint8)
    return frame